TP2: Premier réseau de neurone avec PyTorch
=============

Introduction au sujet
-----

L'objectif de ce sujet est de mettre en place un premier réseau de neurone pour classer des fleurs de la base IRIS.

Le code est à écrire en python3 à la suite des questions dans ce fichier. Vous appuierez soit sur le bouton *run cell*, soit sur les touches *Ctrl-Entrée*, à l’intérieur de la zone de saisie, pour lancer l'exécution de vos commandes. Si la commande est en cours d’exécution une étoile apparaît à côté de la zone de saisie de la commande : In [\*]. Une fois le calcul achevé, l'étoile est remplacée par le numéro du run permettant de retrouver par la suite dans quel ordre ont été lancés chaque bloc.

N'hésitez pas à regarder régulièrement la documentation de ces librairies, des exemples d'utilisation accompagnent généralement l'explication de chaque fonction.

Langage utilisé:
- Python 3: https://docs.python.org/3/

Librairie de math:
- Numpy: https://docs.scipy.org/doc/numpy/reference/
- Scipy: https://docs.scipy.org/doc/scipy/reference/

Librairie d'affichage de données:
- Matplotilb: https://matplotlib.org/contents.html

Librairie Pytorch:
- PyTorch: https: https://pytorch.org/docs/stable/

Commencez par importer les librairies nécessaires au TP.

In [1]:
# Import Torch
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

# Import numpy et matplotlib
import numpy as np
import matplotlib.pyplot as plt

# Import de scikit-learn
import sklearn as sk
import sklearn.datasets
import sklearn.model_selection

%matplotlib inline

2024-05-05 01:39:27.797395: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-05 01:39:28.515739: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-05 01:39:30.289223: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


PyTorch: Premier réseau sur la base IRIS
-----

Nous allons dans ce TP étudier la base *IRIS* et tester un réseau entièrement connecté dessus. 

Commencez par charger la base *IRIS* avec sckit-learn. Vous mettrez les descripteurs des fleurs dans une variable `X` et les labels dans une variable `y`.

In [2]:
from sklearn.datasets import load_iris
X,y = load_iris(return_X_y=True)

In [12]:
# Define feature names
feature_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

# Display the header
header = "   ".join(feature_names + ['species'])
print(header)

# Display each row of the dataset
for i in range(len(X)):
    row_data = "\t\t".join([str(x) for x in X[i]] + [str(y[i])])
    print(row_data)

sepal_length   sepal_width   petal_length   petal_width   species
5.1		3.5		1.4		0.2		0
4.9		3.0		1.4		0.2		0
4.7		3.2		1.3		0.2		0
4.6		3.1		1.5		0.2		0
5.0		3.6		1.4		0.2		0
5.4		3.9		1.7		0.4		0
4.6		3.4		1.4		0.3		0
5.0		3.4		1.5		0.2		0
4.4		2.9		1.4		0.2		0
4.9		3.1		1.5		0.1		0
5.4		3.7		1.5		0.2		0
4.8		3.4		1.6		0.2		0
4.8		3.0		1.4		0.1		0
4.3		3.0		1.1		0.1		0
5.8		4.0		1.2		0.2		0
5.7		4.4		1.5		0.4		0
5.4		3.9		1.3		0.4		0
5.1		3.5		1.4		0.3		0
5.7		3.8		1.7		0.3		0
5.1		3.8		1.5		0.3		0
5.4		3.4		1.7		0.2		0
5.1		3.7		1.5		0.4		0
4.6		3.6		1.0		0.2		0
5.1		3.3		1.7		0.5		0
4.8		3.4		1.9		0.2		0
5.0		3.0		1.6		0.2		0
5.0		3.4		1.6		0.4		0
5.2		3.5		1.5		0.2		0
5.2		3.4		1.4		0.2		0
4.7		3.2		1.6		0.2		0
4.8		3.1		1.6		0.2		0
5.4		3.4		1.5		0.4		0
5.2		4.1		1.5		0.1		0
5.5		4.2		1.4		0.2		0
4.9		3.1		1.5		0.2		0
5.0		3.2		1.2		0.2		0
5.5		3.5		1.3		0.2		0
4.9		3.6		1.4		0.1		0
4.4		3.0		1.3		0.2		0
5.1		3.4		1.5		0.2		0
5.0		3.5		1.3		0.3		0
4.5		2.3		1.3		0.3		0
4.4		3.2		

Vérifiez que le code suivant affiche bien des *True*.

In [2]:
print(X.shape == (150,4))
print(y.shape == (150,))

True
True


Séparez l'ensemble d'apprentissage en deux en utilisant la fonction train_test_split de sckit-learn. Un ensemble d'apprentissage *train* et un ensemble de *test*. Vous prendrez 1/3 des images pour le test.

In [4]:
from sklearn.model_selection import train_test_split
train, test, y_train, y_test = train_test_split(X, y, test_size=(1/3), random_state=42)

Vérifiez les dimensions des données produites:

In [5]:
print(2*len(X)/3,'->',train.shape,y_train.shape)
print(len(X)/3,'->',test.shape,y_test.shape)

100.0 -> (100, 4) (100,)
50.0 -> (50, 4) (50,)


Définissez un classifieur *iris_classifier* correspondant à un réseau entièrement connecté de 3 couches cachées de tailles [10,20,10]. Après chaque couche cachée, vous appliquerez une fonction d'activation de type ReLU. N'oubliez pas la dernière couche de sortie. Vous utiliseriez `torch.nn.Sequential` pour faire cette question.

In [15]:
import torch.nn as nn

# Définir le classifieur iris_classifier
iris_classifier = nn.Sequential(
    nn.Linear(4, 10),  # Couche d'entrée: 4 neurones, sortie: 10 neurones
    nn.ReLU(),          # Fonction d'activation ReLU
    nn.Linear(10, 20),  # Couche cachée: 10 neurones en entrée, 20 neurones en sortie
    nn.ReLU(),          # Fonction d'activation ReLU
    nn.Linear(20, 10),  # Couche cachée: 20 neurones en entrée, 10 neurones en sortie
    nn.ReLU(),          # Fonction d'activation ReLU
    nn.Linear(10, 3),   # Couche de sortie: 10 neurones en entrée, 3 neurones en sortie (classes)
    nn.Softmax(dim=1)   # Fonction d'activation Softmax pour obtenir les probabilités des classes
)

# Afficher l'architecture du classifieur
print(iris_classifier)

Sequential(
  (0): Linear(in_features=4, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=20, bias=True)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=10, bias=True)
  (5): ReLU()
  (6): Linear(in_features=10, out_features=3, bias=True)
  (7): Softmax(dim=1)
)


Définissez un objet `iter_train` permettant de parcourir la base de donnée d'entrainement avec des batchs aléatoires de taille 32. Vous utiliserez les classes `TensorDataset` et `DataLoader` pour cette question.

In [18]:
from torch.utils.data import TensorDataset, DataLoader

# Créer un TensorDataset à partir des données d'entraînement et des étiquettes
train_dataset = TensorDataset(torch.tensor(train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))

# Définir DataLoader pour itérer sur les mini-lots aléatoires de taille 32
iter_train = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Vérifier le nombre total de mini-lots dans l'itérateur
num_batches = len(iter_train)

print("Nombre total de mini-lots dans l'itérateur d'entraînement:", num_batches)

Nombre total de mini-lots dans l'itérateur d'entraînement: 4


Définissez un objet `iter_test` permettant de parcourir la base de donnée de test avec des batchs de taille 10 concervant l'ordre d'origine des exemples.

In [21]:
# Créer un TensorDataset à partir des données de test et des étiquettes
test_dataset = TensorDataset(torch.tensor(test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32))

# Définir DataLoader pour itérer sur les mini-lots de taille 10 tout en conservant l'ordre
iter_test = DataLoader(test_dataset, batch_size=10, shuffle=False)

# Vérifier le nombre total de mini-lots dans l'itérateur
num_batches_test = len(iter_test)
print("Nombre total de mini-lots dans l'itérateur de test:", num_batches_test)

Nombre total de mini-lots dans l'itérateur de test: 5


Définissez une optimiser de type gradient stochastique initialisé avec un taux d'apprentissage de $10^{-2}$.

In [24]:
import torch.optim as optim

# Définir l'optimiseur SGD avec un taux d'apprentissage de 10^-2
learning_rate = 1e-2
optimizer = optim.SGD(iris_classifier.parameters(), lr=learning_rate)

# Vérifier le taux d'apprentissage de l'optimiseur
print("Taux d'apprentissage de l'optimiseur SGD:", optimizer.param_groups[0]['lr'])

Taux d'apprentissage de l'optimiseur SGD: 0.01


Définissze un critère de type cross-entropie qui sera utilisé comme fonction de coût optimisant notre réseau.

In [25]:
import torch.nn as nn

# Définir le critère de perte CrossEntropy
criterion = nn.CrossEntropyLoss()

# Afficher le critère de perte
print("Critère de perte CrossEntropy:", criterion)

Critère de perte CrossEntropy: CrossEntropyLoss()


Effectuez 100 époques d'apprentissage du classifieur `iris_classifier` avec les données de `iter_train`. Vous utiliserez pour celà un algorithme de gradient stochastique avec une fonction de coût de type cross-entropie.

In [26]:
import torch.optim as optim

# Définir l'optimiseur SGD avec un taux d'apprentissage de 10^-2
learning_rate = 1e-2
optimizer = optim.SGD(iris_classifier.parameters(), lr=learning_rate)

# Définir le critère de perte CrossEntropy
criterion = nn.CrossEntropyLoss()

# Nombre d'époques
num_epochs = 100

# Boucle d'apprentissage sur les époques
for epoch in range(num_epochs):
    # Boucle sur les mini-lots dans iter_train
    for inputs, labels in iter_train:
        # Remettre à zéro les gradients
        optimizer.zero_grad()
        
        # Effectuer une propagation avant (forward)
        outputs = iris_classifier(inputs)
        
        # Calculer la perte
        loss = criterion(outputs, torch.argmax(labels, axis=1))
        
        # Effectuer une rétropropagation (backward)
        loss.backward()
        
        # Mettre à jour les poids
        optimizer.step()
    
    # Afficher la perte moyenne pour chaque époque
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Entraînement terminé.")

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

0 /100, loss= 0.8554646968841553
10 /100, loss= 1.0755116939544678
20 /100, loss= 1.0671064853668213
30 /100, loss= 0.9227153658866882
40 /100, loss= 0.9295974969863892
50 /100, loss= 0.5762283802032471
60 /100, loss= 0.479737788438797
70 /100, loss= 0.6177878379821777
80 /100, loss= 0.33995407819747925
90 /100, loss= 0.4962453544139862
99 /100, loss= 0.3136560320854187


Evaluez les performances du réseau appris à la  question précédente sur les données de test de `iter_test`. Pour faire  cette  question vous calculerez dans une boucle le nombre de fois que l'algorithme s'est trompé sur la base de test. Pensez à désactiver le calcul des gradients sur la base de test afin de pas perturbé avec des données de tests de nouveaux apprentissages de votre réseau.

Accuracy sur test: 78.0%


Relancez les lignes effectuant l'apprentissage et l'évaluation. Comment évolue les performances d'apprentissage ? 

Utilisez tensorboard pour visualiser le graphe correspondant à votre réseau et les différentes courbes correspondant à l'apprentissage de ce dernier.

Vous pouvez pour cela:
- Soit installez le plugin tensorboard pour jupyter: pip3 install --user jupyter-tensorboard

Puis vous suivez les informations d'écrit sur la page: https://github.com/lspvic/jupyter_tensorboard
- Soit lancer dans un terminal: tensorboard --logdir=.

Puis vous vous connectez à http://localhost:6006

PyTorch: Définition d'un réseau couche par couche
-----

Nous allons dans cette partie redéfinir le réseau couche par couche.

Définissez une classe `Net` définissant le réseau précédant sans utiliser de `torch.nn.Sequential`. 

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 10)  # Input size: 4, Output size: 10
        self.fc2 = nn.Linear(10, 20)  # Input size: 10, Output size: 20
        self.fc3 = nn.Linear(20, 10)   # Input size: 20, Output size: 10
        self.fc4 = nn.Linear(10, 3)    # Input size: 10, Output size: 3

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=1)
        return x

# Instantiate the model
iris_classifier = Net()

# Print the model architecture
print(iris_classifier)

Net(
  (fc1): Linear(in_features=4, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=3, bias=True)
)


Apprenez ce réseau sur les données d'apprentissage de la base IRIS avec un algorithme de descende de gradient de type AdaGrad dont le taux d'apprentissage est de $10^{-2}$ avec une fonction de coût de type cross-entropie.

0 /200, loss= 1.1917170286178589
100 /200, loss= 0.3672744035720825
200/200, loss= 0.13490986824035645


Testez le réseau que vous venez d'apprendre sur la base de test.

Accuracy sur test: 96.0%


Sauvegardez le modèle que vous venez d'apprendre dans un fichier.

Chargez le réseau que vous venez de sauvegarder et vérifier que les performances sur la base de test n'ont pas changé.

Accuracy sur test: 96.0%


Comparez les graphes des deux méthodes dans tensorboard. Retrouvez-vous les même éléments ? Qu'est ce qui diffère entre les deux versions ? 

Comparaison avec un SVM 
----

En utilisant la librairie sckit-learn et le cours de Machine learning du premier semestre, trouver les performances du meilleur SVM sur ces données et comparer les performances avec celle du réseau de neurone.

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import *



0.98
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        16
           1     1.0000    0.9474    0.9730        19
           2     0.9375    1.0000    0.9677        15

   micro avg     0.9800    0.9800    0.9800        50
   macro avg     0.9792    0.9825    0.9802        50
weighted avg     0.9812    0.9800    0.9801        50

[[16  0  0]
 [ 0 18  1]
 [ 0  0 15]] 0.8 0.74


/home/lechervy/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
